In [1]:
import os
import pandas as pd
import requests
import bs4
import lxml
import functions.web_scrapper_functions as scrpr
import mysql.connector
import json

In [2]:
frag_type_dict = scrpr.get_frag_types()
print("Available fragrance types: \n", frag_type_dict.keys())

Available fragrance types: 
 dict_keys(['Body Mist', 'Car Perfume Diffuser', 'Perfume Oils', 'Our Creations', 'Dahn Al Oud', 'Interior Perfume', 'Musk Oil', 'Perfume Reed Diffuser'])


In [3]:
prod_list = scrpr.prod_list_our_creations(frag_type_dict)
prods = scrpr.prod_info_our_creations(prod_list)

In [4]:
frag_type_dict

{'Body Mist': 'https://www.parfum.ae/body-mist/',
 'Car Perfume Diffuser': 'https://www.parfum.ae/car-perfume-diffuser/',
 'Perfume Oils': 'https://www.parfum.ae/perfume-oils/',
 'Our Creations': 'https://www.parfum.ae/oils/',
 'Dahn Al Oud': 'https://www.parfum.ae/dahn-al-oud/',
 'Interior Perfume': 'https://www.parfum.ae/interior-perfume/',
 'Musk Oil': 'https://www.parfum.ae/musk-oil/',
 'Perfume Reed Diffuser': 'https://www.parfum.ae/essential-oil-diffuser/'}

In [13]:
conn = mysql.connector.connect(
    host = "localhost",
    user = os.environ.get("MYSQL_USERNAME"),
    password = os.environ.get("MYSQL_PASSWORD"),
    database = "parfumae_products"
)

cursor = conn.cursor()

In [14]:

try:
    for key, val in frag_type_dict.items():
        serialized_value = json.dumps(val)

        insert_query = "INSERT INTO fragrance_types (name, url) VALUES (%s, %s)"
        cursor.execute(insert_query, (key, serialized_value))
    
    conn.commit()

except mysql.connector.Error as err:
    print(f"Error: {err}")
    conn.rollback()

cursor.close()
conn.close()

In [6]:
import json
for key, val in frag_type_dict.items():
    print(json.dumps(val))

"https://www.parfum.ae/body-mist/"
"https://www.parfum.ae/car-perfume-diffuser/"
"https://www.parfum.ae/perfume-oils/"
"https://www.parfum.ae/oils/"
"https://www.parfum.ae/dahn-al-oud/"
"https://www.parfum.ae/interior-perfume/"
"https://www.parfum.ae/musk-oil/"
"https://www.parfum.ae/essential-oil-diffuser/"
